# BARONE-ADESI AND WHALEY APPROXIMATION

In [1]:
import pandas as pd 
import numpy as np
from scipy.stats import norm, multivariate_normal

def GBlackScholes(CallPutFlag, S, X, T, r, b, v):
    d1 = (np.log(S / X) + (b + v**2 / 2) * T) / (v * np.sqrt(T))
    d2 = d1 - v * np.sqrt(T)
    
    if CallPutFlag.lower() == "c":
        return S * np.exp((b - r) * T) * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    elif CallPutFlag.lower() == "p":
        return X * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp((b - r) * T) * norm.cdf(-d1)

def Kc(X, T, r, b, v):
    N = 2 * b / v**2
    m = 2 * r / v**2
    q2u = (- (N - 1) + np.sqrt((N - 1)**2 + 4 * m)) / 2
    su = X / (1 - 1 / q2u)
    h2 = -(b * T + 2 * v * np.sqrt(T)) * X / (su - X)
    Si = X + (su - X) * (1 - np.exp(h2))
    K = 2 * r / (v**2 * (1 - np.exp(-r * T)))
    d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * np.sqrt(T))
    Q2 = (- (N - 1) + np.sqrt((N - 1)**2 + 4 * K)) / 2
    LHS = Si - X
    REIS = GBlackScholes("c", Si, X, T, r, b, v) + (1 - np.exp((b - r) * T) * norm.cdf(d1)) * Si / Q2
    bi = np.exp((b - r) * T) * norm.cdf(d1) * (1 - 1 / Q2) + (1 - np.exp((b - r) * T) * norm.cdf(d1) / (v * np.sqrt(T))) / Q2
    E = 1e-07

    while abs(LHS - REIS) / X > E:
        Si = (X + REIS - bi * Si) / (1 - bi)
        d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * np.sqrt(T))
        LHS = Si - X
        REIS = GBlackScholes("c", Si, X, T, r, b, v) + (1 - np.exp((b - r) * T) * norm.cdf(d1)) * Si / Q2
        bi = np.exp((b - r) * T) * norm.cdf(d1) * (1 - 1 / Q2) + (1 - np.exp((b - r) * T) * norm.cdf(d1) / (v * np.sqrt(T))) / Q2

    return Si

def BAWAmericanCallApprox(S, X, T, r, b, v):
    if (b >= r) or (abs(r)>b):
        return GBlackScholes("c", S, X, T, r, b, v)
    else:
        Sk = Kc(X, T, r, b, v)
        N = 2 * b / v**2
        K = 2 * r / (v**2 * (1 - np.exp(-r * T)))
        d1 = (np.log(Sk / X) + (b + v**2 / 2) * T) / (v * np.sqrt(T))
        Q2 = (- (N - 1) + np.sqrt((N - 1)**2 + 4 * K)) / 2
        a2 = (Sk / Q2) * (1 - np.exp((b - r) * T) * norm.cdf(d1))
        
        if S < Sk:
            return GBlackScholes("c", S, X, T, r, b, v) + a2 * (S / Sk)**Q2
        else:
            return S - X

# BJERKSUND AND STENSLAND (1993) APPROXIMATION

In [2]:
def phi(S, T, gamma, h, i, r, b, v):
    lambda_ = (-r + gamma * b + 0.5 * gamma * (gamma - 1) * v**2) * T
    d = -(np.log(S / h) + (b + (gamma - 0.5) * v**2) * T) / (v * np.sqrt(T))
    kappa = 2 * b / (v**2) + (2 * gamma - 1)
    
    return np.exp(lambda_) * S**gamma * (norm.cdf(d) - (i / S)**kappa * norm.cdf(d - 2 * np.log(i / S) / (v * np.sqrt(T))))


def BSAmericanCallApprox1993(S, X, T, r, b, v):
    if b >= r:
        return GBlackScholes("c", S, X, T, r, b, v)
    else:
        Beta = (0.5 - b / v**2) + np.sqrt((b / v**2 - 0.5)**2 + 2 * r / v**2)
        BInfinity = Beta / (Beta - 1) * X
        BO = max(X, r / (r - b) * X)
        h = -(b * T + 2 * v * np.sqrt(T)) * BO / (BInfinity - BO)
        i = BO + (BInfinity - BO) * (1 - np.exp(h))
        Alpha = (i - X) * i**(-Beta)
        
        if S >= i:
            return S - X
        else:
            return Alpha * S**Beta - Alpha * phi(S, T, Beta, i, i, r, b, v) + phi(S, T, 1, i, i, r, b, v) - phi(S, T, 1, X, i, r, b, v) - X * phi(S, T, 0, i, i, r, b, v) + X * phi(S, T, 0, X, i, r, b, v)



# BJERKSUND AND STENSLAND (2002) APPROXIMATION

In [3]:
def phi(S, T, gamma, h, i, r, b, v):
    lambda_ = (-r + gamma * b + 0.5 * gamma * (gamma - 1) * v**2) * T
    d = -(np.log(S / h) + (b + (gamma - 0.5) * v**2) * T) / (v * np.sqrt(T))
    kappa = 2 * b / (v**2) + (2 * gamma - 1)
    phi = np.exp(lambda_) * S**gamma * (norm.cdf(d) - (i / S) ** kappa * norm.cdf(d - 2 * np.log(i / S) / (v * np.sqrt(T))))
    return phi

def CBND(X, y, rho):
    W = np.array([
        [0.17132449237917, 0.0471753363865118, 0.0176140071391521],
        [0.360761573048138, 0.106939325995318, 0.0406014298003869],
        [0.46791393457269, 0.160078328543346, 0.0626720483341091],
        [0, 0.203167426723066, 0.0832767415767048],
        [0, 0.233492536538355, 0.10193011981724],
        [0, 0.249147045813403, 0.118194531961518],
        [0, 0, 0.131688638449177],
        [0, 0, 0.142096109318382],
        [0, 0, 0.149172986472604],
        [0, 0, 0.152753387130726]
    ])

    XX = np.array([
        [-0.932469514203152, -0.981560634246719, -0.993128599185095],
        [-0.661209386466265, -0.904117256370475, -0.963971927277914],
        [-0.238619186083197, -0.769902674194305, -0.912234428251326],
        [0, -0.587317954286617, -0.839116971822219],
        [0, -0.36783149899818, -0.746331906460151],
        [0, -0.125233408511469, -0.636053680726515],
        [0, 0, -0.510867001950827],
        [0, 0, -0.37370608871542],
        [0, 0, -0.227785851141645],
        [0, 0, -0.0765265211334973]
    ])


    if abs(rho) < 0.3:
        NG = 1
        LG = 3
    elif abs(rho) < 0.75:
        NG = 2
        LG = 6
    else:
        NG = 3
        LG = 10

    h = -X
    k = -y
    hk = h * k
    BVN = 0

    if abs(rho) < 0.925:
        if abs(rho) > 0:
            hs = (h * h + k * k) / 2
            asr = np.arcsin(rho)
            for i in range(LG):
                for ISs in [-1, 1]:
                    sn = np.sin(asr * (ISs * XX[i, NG-1] + 1) / 2)
                    BVN = BVN + W[i, NG-1] * np.exp((sn * hk - hs) / (1 - sn * sn))
            BVN = BVN * asr / (4 * np.pi)
        BVN = BVN + norm.cdf(-h) * norm.cdf(-k)
    else:
        if rho < 0:
            k = -k
            hk = -hk
        if abs(rho) < 1:
            Ass = (1 - rho) * (1 + rho)
            A = np.sqrt(Ass)
            bs = (h - k) ** 2
            c = (4 - hk) / 8
            d = (12 - hk) / 16
            asr = -(bs / Ass + hk) / 2
            if asr > -100:
                BVN = A * np.exp(asr) * (1 - c * (bs - Ass) * (1 - d * bs / 5) / 3 + c * d * Ass * Ass / 5)
            if -hk < 100:
                b = np.sqrt(bs)
                BVN = BVN - np.exp(-hk / 2) * np.sqrt(2 * np.pi) * norm.cdf(-b / A) * b * (1 - c * bs * (1 - d * bs / 5) / 3)
            A = A / 2
            for i in range(LG):
                for ISs in [-1, 1]:
                    xs = (A * (ISs * XX[i, NG-1] + 1)) ** 2
                    rs = np.sqrt(1 - xs)
                    asr = -(bs / xs + hk) / 2
                    if asr > -100:
                        BVN = BVN + A * W[i, NG-1] * np.exp(asr) * (norm.cdf(-hk * (1 - rs) / (2 * (1 + rs))) / rs - (1 + c * xs * (1 + d * xs)))
            BVN = -BVN / (2 * np.pi)
        if rho > 0:
            BVN = BVN + norm.cdf(-min(h, k))
        else:
            BVN = -BVN
            if k > h:
                BVN = BVN + norm.cdf(k) - norm.cdf(h)
    return BVN
def ksi(S, T2, gamma, h, I2, I1, t1, r, b, v):
    e1 = (np.log(S / I1) + ((b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1)))
    e2 = (np.log(I2**2 / (S * I1)) + (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e3 = (np.log(S / I1) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e4 = (np.log(I2**2 / (S * I1)) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    
    f1 = (np.log(S / h) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f2 = (np.log(I2**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f3 = (np.log(I1**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f4 = (np.log((S * I1**2) / (h * I2**2)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    rho = np.sqrt(t1 / T2)
    lambda_ = -r + gamma * b + 0.5 * gamma * (gamma - 1) * v**2
    kappa = 2 * b / (v**2) + (2 * gamma - 1)
    

    '''return np.exp(lambda_ * T2) * S**gamma   *(multivariate_normal.cdf([-e1, -f1], cov=[[1, rho], [rho, 1]])   \
                          - (I2 / S)**kappa  * multivariate_normal.cdf([-e2, -f2], cov=[[1, rho], [rho, 1]])   \
                          - (I1 / S)**kappa  * multivariate_normal.cdf([-e3, -f3], cov=[[1, -rho], [-rho, 1]]) \
                          + (I1 / I2)**kappa * multivariate_normal.cdf([-e4, -f4], cov=[[1, -rho], [-rho, 1]]))'''

    return np.exp(lambda_ * T2) * S**gamma   * (CBND(-e1, -f1, rho)    \
                          - (I2 / S)**kappa  *  CBND(-e2, -f2, rho)    \
                          - (I1 / S)**kappa  *  CBND(-e3, -f3, -rho)   \
                          + (I1 / I2)**kappa *  CBND(-e4, -f4, -rho) )

def BSAmericanApprox2002(CallPutFlag, S, X, T, r, b, v):
    if CallPutFlag.lower() == "c":
        return BSAmericanCallApprox2002(S, X, T, r, b, v)
    elif CallPutFlag.lower() == "p":
        return BSAmericanCallApprox2002(X, S, T, r - b, -b, v)

def BSAmericanCallApprox2002(S, X, T, r, b, v):
    t1 = 0.5 * (np.sqrt(5) - 1) * T

    if b >= r:
        return GBlackScholes("c", S, X, T, r, b, v)
    else:
        Beta = (0.5 - b / v**2) + np.sqrt((b / v**2 - 0.5)**2 + 2 * r / v**2)
        BInfinity = Beta / (Beta - 1) * X
        BO = np.maximum(X, r / (r - b) * X)
        h1 = -(b * t1 + 2 * v * np.sqrt(t1)) * X**2 / ((BInfinity - BO) * BO)
        h2 = -(b * T + 2 * v * np.sqrt(T)) * X**2 / ((BInfinity - BO) * BO)
        I1 = BO + (BInfinity - BO) * (1 - np.exp(h1))
        I2 = BO + (BInfinity - BO) * (1 - np.exp(h2))
        alfal = (I1 - X) * I1**(-Beta)
        alfa2 = (I2 - X) * I2**(-Beta)

        if S >= I2:
            return S - X
        else:
            return alfa2 * S**Beta - alfa2 * phi(S, t1, Beta, I2, I2, r, b, v) \
                    + phi(S, t1, 1, I2, I2, r, b, v) \
                    - phi(S, t1, 1, I1, I2, r, b, v) \
                    - X * phi(S, t1, 0, I2, I2, r, b, v) \
                    + X * phi(S, t1, 0, I1, I2, r, b, v) \
                    + alfal * phi(S, t1, Beta, I1, I2, r, b, v) \
                    - alfal * ksi(S, T, Beta, I1, I2, I1, t1, r, b, v) \
                    + ksi(S, T, 1, I1, I2, I1, t1, r, b, v) \
                    - ksi(S, T, 1, X, I2, I1, t1, r, b, v) \
                    - X * ksi(S, T, 0, I1, I2, I1, t1, r, b, v) \
                    + X * ksi(S, T, 0, X, I2, I1, t1, r, b, v)

In [4]:
S, X, T, r, b, v = 110, 100, 0.5, 0.1, 0.0, 0.4
#S, X, T, r, b, v = 42, 40, 0.75, 0.04, -0.04, 0.35
data_dict = {}
for v in np.linspace(0.15,0.4,6):
    data_dict[v] = {
        'GBlackScholes':   GBlackScholes('c', S, X, T, r, b, v),  
        'BAW_Approx':     BAWAmericanCallApprox(S, X, T, r, b, v),
        'BS_1993_Approx':  BSAmericanCallApprox1993(S, X, T, r, b, v),
        'BS_2002_Approx': BSAmericanApprox2002('c', S, X, T, r, b, v),
           }

df = pd.DataFrame(data_dict).T
df


,GBlackScholes,BAW_Approx,BS_1993_Approx,BS_2002_Approx
0.15,10.576858,10.576858,10.782787,11.087510
0.20,11.615697,11.615697,11.803431,12.542865
0.25,12.785679,12.785679,12.969301,14.109521
0.30,14.026528,14.026528,14.212367,15.694390
0.35,15.308600,15.308600,15.499845,17.260626
0.40,16.615819,16.615819,16.814136,18.793139


In [5]:
data_dict = {}
for v in np.linspace(0.15,0.4,6):
    data_dict[v] = {
        'GBlackScholes':   GBlackScholes('p', S, X, T, r , b, v),
        'BAW_Approx':     BAWAmericanCallApprox(X, S, T, r - b, -b, v),
        'BS_1993_Approx':  BSAmericanCallApprox1993(X, S, T, r - b, -b, v),
        'BS_2002_Approx': BSAmericanApprox2002('p', S, X, T, r, b, v),
           }

df = pd.DataFrame(data_dict).T
df

,GBlackScholes,BAW_Approx,BS_1993_Approx,BS_2002_Approx
0.15,1.064564,1.064564,1.067530,3.582586
0.20,2.103403,2.103403,2.111479,5.058655
0.25,3.273385,3.273385,3.288637,6.518656
0.30,4.514234,4.514234,4.538202,7.949285
0.35,5.796306,5.796306,5.830122,9.345128
0.40,7.103525,7.103525,7.148006,10.705372


# AMERICAN PERPETUAL OPTIONS

In [6]:
def AmericanPerpetual(CallPutFlag, S, X, r, b, v):
    y = 0.5 -b/v**2 +np.sqrt((b/v**2-0.5)**2+2*r/v**2)
    if CallPutFlag =='c':
        res = X/(y-1)*((y-1)/y*S/X)**y
    elif CallPutFlag =='p':
        res = X/(1-y)*((y-1)/y*S/X)**y
    return res

In [7]:
X, r, b = 100, 0.1, 0.02
S_list = np.linspace(70,130,7)
v_list = np.linspace(0.1,0.3,5)
A = np.zeros((len(S_list),len(v_list)))
for i, S in enumerate(S_list):
    for j, v in enumerate(v_list):
        A[i,j] = AmericanPerpetual('c', S, X, r, b, v)
pd.DataFrame(A.T, index = v_list, columns=S_list)

,70.0,80.0,90.0,100.0,110.0,120.0,130.0
0.10,4.322907,6.642559,9.702700,13.617394,18.503500,24.480446,31.670046
0.15,6.893273,9.777690,13.308922,17.535834,22.505248,28.262226,34.850281
0.20,9.681153,13.049714,16.981714,21.493067,26.598384,32.311196,38.644124
0.25,12.539248,16.329506,20.613275,25.389268,30.656336,36.413446,42.659653
0.30,15.386148,19.551322,24.151905,29.177438,34.618785,40.467851,46.717388


In [8]:
import numpy as np
from scipy.stats import norm

def CBND(X, y, rho):
    W = np.array([
        [0.17132449237917, 0.0471753363865118, 0.0176140071391521],
        [0.360761573048138, 0.106939325995318, 0.0406014298003869],
        [0.46791393457269, 0.160078328543346, 0.0626720483341091],
        [0, 0.203167426723066, 0.0832767415767048],
        [0, 0.233492536538355, 0.10193011981724],
        [0, 0.249147045813403, 0.118194531961518],
        [0, 0, 0.131688638449177],
        [0, 0, 0.142096109318382],
        [0, 0, 0.149172986472604],
        [0, 0, 0.152753387130726]
    ])

    XX = np.array([
        [-0.932469514203152, -0.981560634246719, -0.993128599185095],
        [-0.661209386466265, -0.904117256370475, -0.963971927277914],
        [-0.238619186083197, -0.769902674194305, -0.912234428251326],
        [0, -0.587317954286617, -0.839116971822219],
        [0, -0.36783149899818, -0.746331906460151],
        [0, -0.125233408511469, -0.636053680726515],
        [0, 0, -0.510867001950827],
        [0, 0, -0.37370608871542],
        [0, 0, -0.227785851141645],
        [0, 0, -0.0765265211334973]
    ])


    if abs(rho) < 0.3:
        NG = 1
        LG = 3
    elif abs(rho) < 0.75:
        NG = 2
        LG = 6
    else:
        NG = 3
        LG = 10

    h = -X
    k = -y
    hk = h * k
    BVN = 0

    if abs(rho) < 0.925:
        if abs(rho) > 0:
            hs = (h * h + k * k) / 2
            asr = np.arcsin(rho)
            for i in range(LG):
                for ISs in [-1, 1]:
                    sn = np.sin(asr * (ISs * XX[i, NG-1] + 1) / 2)
                    BVN = BVN + W[i, NG-1] * np.exp((sn * hk - hs) / (1 - sn * sn))
            BVN = BVN * asr / (4 * np.pi)
        BVN = BVN + norm.cdf(-h) * norm.cdf(-k)
    else:
        if rho < 0:
            k = -k
            hk = -hk
        if abs(rho) < 1:
            Ass = (1 - rho) * (1 + rho)
            A = np.sqrt(Ass)
            bs = (h - k) ** 2
            c = (4 - hk) / 8
            d = (12 - hk) / 16
            asr = -(bs / Ass + hk) / 2
            if asr > -100:
                BVN = A * np.exp(asr) * (1 - c * (bs - Ass) * (1 - d * bs / 5) / 3 + c * d * Ass * Ass / 5)
            if -hk < 100:
                b = np.sqrt(bs)
                BVN = BVN - np.exp(-hk / 2) * np.sqrt(2 * np.pi) * norm.cdf(-b / A) * b * (1 - c * bs * (1 - d * bs / 5) / 3)
            A = A / 2
            for i in range(LG):
                for ISs in [-1, 1]:
                    xs = (A * (ISs * XX[i, NG-1] + 1)) ** 2
                    rs = np.sqrt(1 - xs)
                    asr = -(bs / xs + hk) / 2
                    if asr > -100:
                        BVN = BVN + A * W[i, NG-1] * np.exp(asr) * (norm.cdf(-hk * (1 - rs) / (2 * (1 + rs))) / rs - (1 + c * xs * (1 + d * xs)))
            BVN = -BVN / (2 * np.pi)
        if rho > 0:
            BVN = BVN + norm.cdf(-min(h, k))
        else:
            BVN = -BVN
            if k > h:
                BVN = BVN + norm.cdf(k) - norm.cdf(h)
    return BVN

def ArcSin(X):
    if abs(X) == 1:
        return np.sign(X) * np.pi / 2
    else:
        return np.arctan(X / np.sqrt(1 - X**2))


def mnv_p(S, T2, gamma, h, I2, I1, t1, r, b, v):
    e1 = (np.log(S / I1) - (-(b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1)))
    e2 = (np.log(I2**2 / (S * I1)) + (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e3 = (np.log(S / I1) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e4 = (np.log(I2**2 / (S * I1)) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    f1 = (np.log(S / h) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f2 = (np.log(I2**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f3 = (np.log(I1**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f4 = (np.log((S * I1**2) / (h * I2**2)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    rho = np.sqrt(t1 / T2)
    lambda_ = -r + gamma * b + 0.5 * gamma * (gamma - 1) * v**2
    kappa = 2 * b / (v**2) + (2 * gamma - 1)
    return multivariate_normal.cdf([-e1, -f1], cov=[[1, rho], [rho, 1]])
    
def mnv_v(S, T2, gamma, h, I2, I1, t1, r, b, v):
    e1 = (np.log(S / I1) - (-(b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1)))
    e2 = (np.log(I2**2 / (S * I1)) + (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e3 = (np.log(S / I1) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    e4 = (np.log(I2**2 / (S * I1)) - (b + (gamma - 0.5) * v**2) * t1) / (v * np.sqrt(t1))
    f1 = (np.log(S / h) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f2 = (np.log(I2**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f3 = (np.log(I1**2 / (S * h)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    f4 = (np.log((S * I1**2) / (h * I2**2)) + (b + (gamma - 0.5) * v**2) * T2) / (v * np.sqrt(T2))
    rho = np.sqrt(t1 / T2)
    lambda_ = -r + gamma * b + 0.5 * gamma * (gamma - 1) * v**2
    kappa = 2 * b / (v**2) + (2 * gamma - 1)
    
    return CBND(-e1, -f1, rho) 
   

def t2002p(S, X, T, r, b, v):
    t1 = 0.5 * (np.sqrt(5) - 1) * T

    if b >= r:
        return GBlackScholes("c", S, X, T, r, b, v)
    else:
        Beta = (0.5 - b / v**2) + np.sqrt((b / v**2 - 0.5)**2 + 2 * r / v**2)
        BInfinity = Beta / (Beta - 1) * X
        BO = max(X, r / (r - b) * X)
        h1 = -(b * t1 + 2 * v * np.sqrt(t1)) * X**2 / ((BInfinity - BO) * BO)
        h2 = -(b * T + 2 * v * np.sqrt(T)) * X**2 / ((BInfinity - BO) * BO)
        I1 = BO + (BInfinity - BO) * (1 - np.exp(h1))
        I2 = BO + (BInfinity - BO) * (1 - np.exp(h2))
        alfal = (I1 - X) * I1**(-Beta)
        alfa2 = (I2 - X) * I2**(-Beta)

        if S >= I2:
            return S - X
        else:
            return mnv_p(S, T, Beta, I1, I2, I1, t1, r, b, v) 
        
def t2002v(S, X, T, r, b, v):
    t1 = 0.5 * (np.sqrt(5) - 1) * T

    if b >= r:
        return GBlackScholes("c", S, X, T, r, b, v)
    else:
        Beta = (0.5 - b / v**2) + np.sqrt((b / v**2 - 0.5)**2 + 2 * r / v**2)
        BInfinity = Beta / (Beta - 1) * X
        BO = max(X, r / (r - b) * X)
        h1 = -(b * t1 + 2 * v * np.sqrt(t1)) * X**2 / ((BInfinity - BO) * BO)
        h2 = -(b * T + 2 * v * np.sqrt(T)) * X**2 / ((BInfinity - BO) * BO)
        I1 = BO + (BInfinity - BO) * (1 - np.exp(h1))
        I2 = BO + (BInfinity - BO) * (1 - np.exp(h2))
        alfal = (I1 - X) * I1**(-Beta)
        alfa2 = (I2 - X) * I2**(-Beta)

        if S >= I2:
            return S - X
        else:
            return mnv_v(S, T, Beta, I1, I2, I1, t1, r, b, v) 

S, X, T, r, b, v = 110, 100, 0.5, 0.1, 0.0, 0.25
print(t2002p(S, X, T, r, b, v), t2002v(S, X, T, r, b, v))

0.4037530752290547 0.4037530752290547


In [9]:
t2002p(S, X, T, r, b, v)==t2002v(S, X, T, r, b, v)

True